## Question 1
* My first question is about the input define for function in general. Because after I run the nnmf function, the error: X undefined shows up. Below, the question is not about the main body of the function. I just want to show the error message here.

In [1]:
function nnmf(
 X::Matrix{Float64}, 
 r::Int;
 maxiter::Int=2, 
 tol::eltype(X)=1e-4,
 V::Matrix{eltype(X)}=rand(size(X, 1), r),
 W::Matrix{eltype(X)}=rand(r, size(X, 2))
 )  
    # initilize two matrices 
    V_new = zeros(size(V))
    W_new = zeros(size(W))
    
    for t in 1:maxiter # stop after 1000 iterations
        # update v by element
        for k in 1:r, i in 1:size(X, 1)
            # initialize float number for denomenator
            denomenator = 0.0
            # update denomenator 
            for j in 1:size(X, 2)
                denomenator += vecdot(V[i, :], W[:, j]) * W[k, j]
            end
            # update element V_ik
            V_new[i, k] = V[i, k] * vecdot(X[i, :], W[k, :]) / denomenator 
        end


        
        # update W by element
        for j in 1:size(X, 2), k in 1:r
            # initialize float number for denomenator
            denomenator = 0.0
            # update denomenator 
            for i in 1:size(X, 1)
                denomenator += vecdot(V_new[i, :], W[:, j]) * V_new[i, k]
            end
            # update element W_kj
            W_new[k, j] = W[k, j] * vecdot(X[:, j], V_new[:, k]) / denomenator 
        end
        
        # update V W 
        V = copy(V_new)
        W = copy(W_new) 
        
        # calculate Frobenius norm after each iteration
        norm = 0.0
        for j = 1:size(X, 2), i = 1:size(X, 1)
            norm += (X[i, j] - vecdot(V[i, :], W[:, j]))^2 
        end
    
        # exit loop if norm less than tolerance
        if norm <= tol
            exit 
        end
    end      
    
    # Output
    return V, W
end
    
    

LoadError: [91mUndefVarError: X not defined[39m

* Below I try to remove the main body of the function, but the error still shows up. Probably the problem is `V::Matrix{eltype(X)}=rand(size(X, 1), r)` is used X as input for matrix type, but we not acutually having X right now. But I am not sure how to fix that.

In [2]:
function nnmf(
 X::Matrix{Float64}, 
 r::Int;
 maxiter::Int=2, 
 tol::eltype(X)=1e-4,
 V::Matrix{eltype(X)}=rand(size(X, 1), r),
 W::Matrix{eltype(X)}=rand(r, size(X, 2))
 )  
    # simple main body to check the function
    V = zeros(1, 1)
    W = zeros(1, 1)    
    
    # Output
    return V, W
end
    
    

LoadError: [91mUndefVarError: X not defined[39m

* In order to run the function, I remove the tol, V, W input and manually import the matrix in the beginning of the function. But I don't think this is the right way to solve the problem. 

In [10]:
function nnmf_1(
 X::Matrix, 
 r::Int;
 maxiter::Int = 1000, 
 tol::Float64 = 1e-4
 )  
    # readin matrix V0
    path_V0 = "http://Hua-Zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/V0.txt"
    V0 = readdlm(download(path_V0), ' ')

    # readin matrix W0
    path_W0 = "http://Hua-Zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/W0.txt"
    W0 = readdlm(download(path_W0), ' ')

    V = V0[:, 1:r]
    W = W0[1:r, :]
    
    
    # input V W 
    #V = rand(size(X, 1), r)
    #W = rand(r, size(X, 2))
    
    # initilize two matrices 
    V_new = zeros(size(V))
    W_new = zeros(size(W))
    
    for t in 1:maxiter # stop after 1000 iterations
        # update v by element
        for k in 1:r, i in 1:size(X, 1)
            # initialize float number for denomenator
            denomenator = 0.0
            # update denomenator 
            for j in 1:size(X, 2)
                denomenator += dot(V[i, :], W[:, j]) * W[k, j]
            end
            # update element V_ik
            V_new[i, k] = V[i, k] * vecdot(X[i, :], W[k, :]) / denomenator 
        end


        
        # update W by element
        for j in 1:size(X, 2), k in 1:r
            # initialize float number for denomenator
            denomenator = 0.0
            # update denomenator 
            for i in 1:size(X, 1)
                denomenator += vecdot(V_new[i, :], W[:, j]) * V_new[i, k]
            end
            # update element W_kj
            W_new[k, j] = W[k, j] * vecdot(X[:, j], V_new[:, k]) / denomenator 
        end
        
        # update V W 
        V = copy(V_new)
        W = copy(W_new) 
        
        # calculate Frobenius norm after each iteration
        norm = 0.0
        for j = 1:size(X, 2), i = 1:size(X, 1)
            norm += (X[i, j] - vecdot(V[i, :], W[:, j]))^2 
        end
    
        # exit loop if norm less than tolerance
        if norm <= tol
            exit 
        end
    end      
    
    # Output
    return V, W
end
    
    

nnmf_1 (generic function with 1 method)

* **This part is related to the function, if you don't have time, I can talk to you on Monday during office hour**
* Another thing I still did not figure out is my function takes forever to run. I change the maxiter = 2, there are some results. But after I change back to 1000, it does not work. Below shows the result for r = 10. May I have some hint for this problem?
* After I use `@time` to see the memory allocation, I see a big problem here. For this algorithm, I think I have to initilize two matrices for updating V, W, because there is a $b_{ij}$ part, which needs the infomation from the previous V matrix. How should I improve the function?
* Below I try profiling the code, the line with the most count number is line -1.


In [13]:
# readin matrix X
path_X = "http://Hua-Zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/nnmf-2429-by-361-face.txt"
X = readdlm(download(path_X), ' ')

@time nnmf(X,10, maxiter = 2, tol = 10.0^(-4))
@time nnmf(X,10, maxiter = 2, tol = 10.0^(-4))


 14.474627 seconds (340.27 M allocations: 15.496 GiB, 8.40% gc time)
 13.940056 seconds (340.27 M allocations: 15.496 GiB, 7.70% gc time)


([0.0597217 0.0189092 … 0.0457883 0.0568525; 0.0283231 0.0622263 … 0.0130065 0.0762958; … ; 0.0846051 0.0682364 … 0.0143376 0.0289061; 0.0997003 0.0963602 … 0.0815733 0.050848], [0.238528 0.21145 … 0.416089 0.206551; 0.254568 0.095235 … 0.0720512 0.310289; … ; 0.232507 0.148173 … 0.232909 0.184989; 0.141977 0.323308 … 0.0951747 0.281921])

In [14]:
Profile.clear()
@profile nnmf(X,10, maxiter = 2, tol = 10.0^(-4))
Profile.print(format=:flat)

 Count File                        Line Function                               
  5710 .\<missing>                   -1 anonymous                              
    75 .\In[3]                        9 #nnmf#3(::Int64, ::Float64, ::Funct... 
    19 .\In[3]                       13 #nnmf#3(::Int64, ::Float64, ::Funct... 
    14 .\In[3]                       15 #nnmf#3(::Int64, ::Float64, ::Funct... 
     3 .\In[3]                       33 #nnmf#3(::Int64, ::Float64, ::Funct... 
  2371 .\In[3]                       34 #nnmf#3(::Int64, ::Float64, ::Funct... 
    89 .\In[3]                       37 #nnmf#3(::Int64, ::Float64, ::Funct... 
     7 .\In[3]                       47 #nnmf#3(::Int64, ::Float64, ::Funct... 
  2653 .\In[3]                       48 #nnmf#3(::Int64, ::Float64, ::Funct... 
    40 .\In[3]                       51 #nnmf#3(::Int64, ::Float64, ::Funct... 
   439 .\In[3]                       61 #nnmf#3(::Int64, ::Float64, ::Funct... 
     5 .\abstractarray.jl           342 

   315 .\multidimensional.jl        519 _unsafe_getindex!                      
  1977 .\multidimensional.jl        506 _unsafe_getindex(::IndexLinear, ::A... 
    39 .\multidimensional.jl        494 macro expansion                        
  2430 .\multidimensional.jl        495 macro expansion                        
    26 .\multidimensional.jl        509 macro expansion                        
  1461 .\multidimensional.jl        511 macro expansion                        
   132 .\multidimensional.jl        512 macro expansion                        
   315 .\multidimensional.jl        513 macro expansion                        
     1 .\multidimensional.jl        523 macro expansion                        
   314 .\multidimensional.jl        525 macro expansion                        
   156 .\multidimensional.jl        527 macro expansion                        
    12 .\multidimensional.jl        469 to_indices                             
     5 .\multidimensional.jl        476 

In [ ]:
# readin matrix X
path_X = "http://Hua-Zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/nnmf-2429-by-361-face.txt"
X = readdlm(download(path_X), ' ')

@time nnmf(X,10, maxiter = 1000, tol = 10.0^(-4))
@time nnmf(X,10, maxiter = 1000, tol = 10.0^(-4))

### Question 2
* Another trial question related is the rule of 80 character margin. I want to control the width limit for the file path. After I break down the file path into two lines, Julia will add `\n` to the String, which resulted in the failure of reading the file. I guess I can write two strings and then combine them to solve the problem. But is there a better way than that? Below show the file path.

In [6]:
# add package for visulizating image
#Pkg.add("ImageView")
#Pkg.add("Images")
#Pkg.add("TestImages")
#Pkg.build("Cairo")
#Pkg.pin("Cairo", v"0.4.0")
#Pkg.update()
using ImageView, Images

# readin matrix file
path_X = "http://Hua-Zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/nnmf-2429-by-361-face.txt"

X = readdlm(download(path_X), ' ')

# show first three face images
for i in 1:3
    # extract one row pixels to form one face image
    img = reshape(X[1, :], 19, 19)
    imshow(img)
end
    


* Appreiciate your time on reading the long file. Hope you enjoy your day.